In [53]:
import pandas as pd
import mariadb
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from autogluon import tabular
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML
from tqdm import tqdm
import bisect
import numpy as np


conn  = mariadb.connect(
    user='team3', 
    password='dbdb', 
    database='team3db', 
    host='localhost',
    port=3306
)

# 위 커넥션 정보와 동일하게 입력
engine = create_engine("mysql://{user}:{pw}@{host}/{db}".format(user='team3', pw='dbdb',host='localhost', db='team3db'))

In [54]:
select_sql = "SELECT * FROM train"
train = pd.read_sql_query(select_sql, engine,index_col='index')
train

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,LENGTH,SHIPMANAGER,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE,CI_HOUR
index,,,,,,,,,,,,,,,,,,,,,
0,TRAIN_000000,SG,GIW5,Container,30.881018,2018-12-17 21:29,Z618338,30.0,24,24300,...,180.0,CQSB78,Panama,-0.527582,-0.075277,27.637513,2.014493,5,0.002615,3.450000
1,TRAIN_000001,IN,UJM2,Bulk,0.000000,2014-09-23 6:59,X886125,30.0,13,35900,...,180.0,SPNO34,Marshall Islands,2.244962,0.213273,28.263135,3.102933,12,0.000217,0.000000
2,TRAIN_000002,CN,EUC8,Container,0.000000,2015-02-03 22:00,T674582,50.0,12,146000,...,370.0,FNPK22,Malta,-1.050854,-0.122173,17.651842,3.210685,6,0.001614,0.000000
3,TRAIN_000003,JP,ZAG4,Container,0.000000,2020-01-17 4:02,Y847238,20.0,18,6910,...,120.0,PBZV77,Bahamas,-3.180000,-1.610000,6.700000,2.629350,13,0.000356,0.000000
4,TRAIN_000004,SG,GIW5,Container,27.037650,2020-01-26 7:51,A872328,50.0,10,116000,...,300.0,GUCE76,Liberia,-0.330000,-3.280000,25.600000,2.495953,15,0.002615,253.554444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391928,TRAIN_391934,JP,QYY1,Container,0.000000,2017-06-06 5:02,Y375615,20.0,27,6820,...,110.0,KEJZ24,"China, People's Republic Of",0.836331,-0.767929,17.033054,2.157586,14,0.000552,0.000000
391929,TRAIN_391935,SG,GIW5,Bulk,5.884603,2019-10-16 0:36,K635567,10.0,12,3160,...,80.0,JLTM64,Vietnam,-0.660000,0.970000,27.300000,1.253491,8,0.002615,144.061389
391930,TRAIN_391936,US,QGN3,Bulk,70.660241,2021-03-23 22:35,J284147,30.0,8,60300,...,200.0,YERJ68,Singapore,-3.440000,7.990000,21.100000,4.766257,18,0.000155,41.482222


In [55]:
select_sql = "SELECT * FROM test"
test = pd.read_sql_query(select_sql, engine,index_col='index')
test

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,GT,LENGTH,SHIPMANAGER,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE
index,,,,,,,,,,,,,,,,,,,,,
0,TEST_000000,SG,GIW5,Container,1.826589,2020-06-18 11:58,K322654,50.0,18,117000,...,109000,340.0,KQMD74,Panama,0.37,1.63,27.1,1.587063,19,0.002615
1,TEST_000001,CN,WEY7,Cargo,25.399386,2021-05-26 22:20,E376681,10.0,13,3810,...,2560,80.0,LBYS27,Belize,-2.79,-2.33,14.2,2.663972,6,0.001028
2,TEST_000002,CN,NGG6,Container,111.079467,2019-12-16 0:09,F811111,20.0,26,10900,...,8440,140.0,EKFV15,Singapore,0.04,-4.91,9.3,3.255315,8,0.001743
3,TEST_000003,CA,FFM2,Bulk,9.175258,2015-11-16 5:30,A737561,30.0,9,55800,...,31500,190.0,MASW32,Panama,NaN,NaN,NaN,NaN,2,0.000182
4,TEST_000004,JP,QYY1,Container,0.000000,2018-10-24 1:11,A827175,30.0,19,39800,...,33000,220.0,SIEL54,Liberia,NaN,NaN,NaN,NaN,10,0.000552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220486,TEST_220486,BR,TMW2,Bulk,21.712733,2017-04-04 15:53,E731427,60.0,15,298000,...,151000,330.0,UWHE18,Japan,NaN,NaN,NaN,NaN,13,0.000080
220487,TEST_220487,CA,GRQ5,Bulk,4.870490,2016-08-22 16:22,Q434522,30.0,9,80500,...,42700,220.0,BBXV46,Liberia,NaN,NaN,NaN,NaN,13,0.000039
220488,TEST_220488,JP,VYJ1,Tanker,17.068286,2022-07-10 14:53,U148516,10.0,14,1200,...,500,60.0,DYUA26,Japan,-2.87,1.22,27.4,2.405268,23,0.000264


In [56]:

categorical_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']
encoders = {}

for feature in tqdm(categorical_features, desc="Encoding features"):
    le = LabelEncoder()
    train[feature] = le.fit_transform(train[feature].astype(str))
    le_classes_set = set(le.classes_)
    test[feature] = test[feature].map(lambda s: '-1' if s not in le_classes_set else s)
    le_classes = le.classes_.tolist()
    bisect.insort_left(le_classes, '-1')
    le.classes_ = np.array(le_classes)
    test[feature] = le.transform(test[feature].astype(str))
    encoders[feature] = le

Encoding features: 100%|██████████| 6/6 [00:01<00:00,  4.02it/s]


In [57]:
train.columns

Index(['SAMPLE_ID', 'ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'DIST', 'ATA',
       'ID', 'BREADTH', 'BUILT', 'DEADWEIGHT', 'DEPTH', 'DRAUGHT', 'GT',
       'LENGTH', 'SHIPMANAGER', 'FLAG', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE',
       'BN', 'ATA_LT', 'PORT_SIZE', 'CI_HOUR'],
      dtype='object')

In [58]:
test.columns

Index(['SAMPLE_ID', 'ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'DIST', 'ATA',
       'ID', 'BREADTH', 'BUILT', 'DEADWEIGHT', 'DEPTH', 'DRAUGHT', 'GT',
       'LENGTH', 'SHIPMANAGER', 'FLAG', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE',
       'BN', 'ATA_LT', 'PORT_SIZE'],
      dtype='object')

In [59]:
train.drop(columns='SAMPLE_ID',inplace=True)
test.drop(columns='SAMPLE_ID',inplace=True)

In [60]:
train['ATA'] = pd.to_datetime(train['ATA'])
test['ATA'] = pd.to_datetime(test['ATA'])

for df in [train, test]:
    df['year'] = df['ATA'].dt.year
    df['weekday'] = df['ATA'].dt.weekday

train.drop(columns='ATA', inplace=True)
test.drop(columns='ATA', inplace=True)

In [61]:
X_train = train.drop(columns='CI_HOUR')
y_train = train['CI_HOUR']

In [62]:
automl = AutoML(
    algorithms=["CatBoost", "Xgboost", "LightGBM", "Random Forest",'Neural Network'],
    mode="Compete",
    ml_task="regression",
    eval_metric='mae',
    random_state=42,
    validation_strategy={
    "validation_type": "kfold",
    "k_folds": 5,
    "shuffle": True,
    "random_seed": 42
},
    total_time_limit=None,
    model_time_limit=None
)

In [63]:
train.drop(columns=['ID', 'SHIPMANAGER', 'BN', 'ATA_LT'], inplace=True)
test.drop(columns=['ID', 'SHIPMANAGER', 'BN', 'ATA_LT'], inplace=True)

In [64]:
X_train = train.drop(columns='CI_HOUR')
y_train = train['CI_HOUR']

In [65]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(X_train)

In [66]:
automl = AutoML(
    algorithms=["CatBoost", "Xgboost", "LightGBM", "Random Forest",'Neural Network'],
    mode="Compete",
    ml_task="regression",
    eval_metric='mae',
    random_state=42,
    validation_strategy={
    "validation_type": "kfold",
    "k_folds": 5,
    "shuffle": True,
    "random_seed": 42
},
    total_time_limit=None,
    model_time_limit=None
)

In [67]:
automl.fit(train_scaled, y_train)

AutoML directory: AutoML_5
The task is regression with evaluation metric mae
AutoML will use algorithms: ['CatBoost', 'Xgboost', 'LightGBM', 'Random Forest', 'Neural Network']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 5 models
1_Default_LightGBM mae 54.087308 trained in 32.75 seconds
2_Default_Xgboost mae 55.335 trained in 41.3 seconds
3_Default_CatBoost mae 56.02315 trained in 60.7 seconds
4_Default_NeuralNetwork mae 65.136634 trained in 213.77 seconds
5_Default_RandomForest mae 62.009285 trained in 81.23 seconds
* Step not_so_random will try to check up to 45 models
15_LightGBM mae 56.318351 trained i

AutoML(algorithms=['CatBoost', 'Xgboost', 'LightGBM', 'Random Forest',
                   'Neural Network'],
       eval_metric='mae', ml_task='regression', mode='Compete', random_state=42,
       total_time_limit=None,
       validation_strategy={'k_folds': 5, 'random_seed': 42, 'shuffle': True,
                            'validation_type': 'kfold'})

In [70]:
X_train

,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,GT,LENGTH,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,PORT_SIZE,year,weekday
index,,,,,,,,,,,,,,,,,,
0,17,21,2,30.881018,30.0,24,24300,10.0,10.0,16700,180.0,89,-0.527582,-0.075277,27.637513,0.002615,2018,0
1,7,80,0,0.000000,30.0,13,35900,10.0,10.0,23500,180.0,74,2.244962,0.213273,28.263135,0.000217,2014,1
2,4,14,2,0.000000,50.0,12,146000,30.0,20.0,140000,370.0,73,-1.050854,-0.122173,17.651842,0.001614,2015,1
3,8,100,2,0.000000,20.0,18,6910,10.0,10.0,5400,120.0,5,-3.180000,-1.610000,6.700000,0.000356,2020,4
4,17,21,2,27.037650,50.0,10,116000,20.0,10.0,96600,300.0,67,-0.330000,-3.280000,25.600000,0.002615,2020,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391928,8,66,2,0.000000,20.0,27,6820,10.0,10.0,4820,110.0,20,0.836331,-0.767929,17.033054,0.000552,2017,1
391929,17,21,0,5.884603,10.0,12,3160,10.0,10.0,1600,80.0,124,-0.660000,0.970000,27.300000,0.002615,2019,2
391930,21,61,0,70.660241,30.0,8,60300,20.0,10.0,34800,200.0,102,-3.440000,7.990000,21.100000,0.000155,2021,1


In [72]:
X_train.columns

Index(['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'DIST', 'BREADTH', 'BUILT',
       'DEADWEIGHT', 'DEPTH', 'DRAUGHT', 'GT', 'LENGTH', 'FLAG', 'U_WIND',
       'V_WIND', 'AIR_TEMPERATURE', 'PORT_SIZE', 'year', 'weekday'],
      dtype='object')

In [73]:
test.columns

Index(['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'DIST', 'BREADTH', 'BUILT',
       'DEADWEIGHT', 'DEPTH', 'DRAUGHT', 'GT', 'LENGTH', 'FLAG', 'U_WIND',
       'V_WIND', 'AIR_TEMPERATURE', 'PORT_SIZE', 'year', 'weekday'],
      dtype='object')

In [82]:
pred = automl.predict(test)

2024-04-04 08:36:16,642 supervised.exceptions ERROR Missing column: feature_1 in input data. Cannot predict


AutoMLException: Missing column: feature_1 in input data. Cannot predict

In [74]:
test_scaled = scaler.fit_transform(test)
X_train = pd.DataFrame(train_scaled, columns = X_train.columns)
test = pd.DataFrame(test_scaled, columns = test.columns)

In [75]:
submit = pd.read_csv('./data/sample_submission.csv')
submit['CI_HOUR'] = pred
submit.to_csv('./csv/submit.csv', index=False)

NameError: name 'pred' is not defined

In [77]:
X_train = train.drop(columns=['CI_HOUR','V_WIND','ARI_PO'])
Y_train = train['CI_HOUR']


In [81]:

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(X_train)
test_scaled = scaler.transform(test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- ARI_PO
- V_WIND


In [ ]:
X_train = pd.DataFrame(train_scaled, columns = X_train.columns)
test = pd.DataFrame(test_scaled, columns = test.columns)

In [ ]:
automl.fit(X_train, Y_train)
pred = automl.predict(test)
submit = pd.read_csv('./data/sample_submission.csv')
submit['CI_HOUR'] = pred
submit.to_csv('./csv/submit2.csv', index=False)